In [106]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# Lab | Natural Language Processing
### SMS: SPAM or HAM

### Let's prepare the environment

In [107]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer

- Read Data for the Fraudulent Email Kaggle Challenge
- Reduce the training set to speead up development. 

In [108]:
## Read Data for the Fraudulent Email Kaggle Challenge
data = pd.read_csv("../data/kg_train.csv",encoding='latin-1')

# Reduce the training set to speed up development. 
# Modify for final system
data = data.head(1000)
print(data.shape)
data.fillna("",inplace=True)

(1000, 2)


### Let's divide the training and test set into two partitions

In [109]:
# Your code
data


,text,label
0,"DEAR SIR, STRICTLY A PRIVATE BUSINESS PROPOSAL...",1
1,Will do.,0
2,Nora--Cheryl has emailed dozens of memos about...,0
3,Dear Sir=2FMadam=2C I know that this proposal ...,1
4,fyi,0
...,...,...
995,So what's the latest? It sounds contradictory ...,0
996,"TRANSFER OF 36,759,000.00 MILLION POUNDS TO YO...",1
997,Barb I will call to explain. Are you back in t...,0
998,Yang on travelNot free tonite.May work tomorrow,0


In [110]:
X = data.drop(columns="label")
y = data["label"]

In [111]:
print(X)
print(y)

                                                  text
0    DEAR SIR, STRICTLY A PRIVATE BUSINESS PROPOSAL...
1                                             Will do.
2    Nora--Cheryl has emailed dozens of memos about...
3    Dear Sir=2FMadam=2C I know that this proposal ...
4                                                  fyi
..                                                 ...
995  So what's the latest? It sounds contradictory ...
996  TRANSFER OF 36,759,000.00 MILLION POUNDS TO YO...
997  Barb I will call to explain. Are you back in t...
998    Yang on travelNot free tonite.May work tomorrow
999  sbwhoeopSunday February 21 2010 7:42 PMHShaunH...

[1000 rows x 1 columns]
0      1
1      0
2      0
3      1
4      0
      ..
995    0
996    1
997    0
998    0
999    0
Name: label, Length: 1000, dtype: int64


In [112]:
from sklearn.model_selection import train_test_split

In [113]:
X_train,X_test , y_train,y_test =  train_test_split(X,y,test_size=0.2,random_state=42)

## Data Preprocessing

In [114]:
import nltk
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\JITHIN\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [115]:
import string
from nltk.corpus import stopwords
print(string.punctuation)
print(stopwords.words("english")[100:110])
from nltk.stem.snowball import SnowballStemmer
snowball = SnowballStemmer('english')

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
['needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on']


## Now, we have to clean the html code removing words

- First we remove inline JavaScript/CSS
- Then we remove html comments. This has to be done before removing regular tags since comments can contain '>' characters
- Next we can remove the remaining tags

In [116]:
# Your code
from bs4 import BeautifulSoup, Comment
import re



def clean_html(text):
    text = str(text)

    soup = BeautifulSoup(text, "html.parser")
    for tag in soup(["script", "style"]):
        tag.decompose()

    for element in soup(text=lambda t: isinstance(t, Comment)):
        element.extract()

    cleaned = soup.get_text(" ", strip=True)
    cleaned = re.sub(r"\s+", " ", cleaned).strip()

    return cleaned

data["clean_text"] = data["text"].apply(clean_html)



C:\Users\JITHIN\AppData\Local\Temp\ipykernel_24184\127247324.py:10: MarkupResemblesLocatorWarning: The input passed in on this line looks more like a URL than HTML or XML.

If you meant to use Beautiful Soup to parse the web page found at a certain URL, then something has gone wrong. You should use an Python package like 'requests' to fetch the content behind the URL. Once you have the content as a string, you can feed that string into Beautiful Soup.

However, if you want to parse some data that happens to look like a URL, then nothing has gone wrong: you are using Beautiful Soup correctly, and this warning is spurious and can be filtered. To make this warning go away, run this code before calling the BeautifulSoup constructor:

    from bs4 import MarkupResemblesLocatorWarning
    import warnings

    warnings.filterwarnings("ignore", category=MarkupResemblesLocatorWarning)
    
  soup = BeautifulSoup(text, "html.parser")


In [117]:
data = data.drop(columns= ["text"])

In [118]:
data.rename(columns={"clean_text": "text"}, inplace= True)

In [119]:
data

,label,text
0,1,"DEAR SIR, STRICTLY A PRIVATE BUSINESS PROPOSAL..."
1,0,Will do.
2,0,Nora--Cheryl has emailed dozens of memos about...
3,1,Dear Sir=2FMadam=2C I know that this proposal ...
4,0,fyi
...,...,...
995,0,So what's the latest? It sounds contradictory ...
996,1,"TRANSFER OF 36,759,000.00 MILLION POUNDS TO YO..."
997,0,Barb I will call to explain. Are you back in t...
998,0,Yang on travelNot free tonite.May work tomorrow


- Remove all the special characters
    
- Remove numbers
    
- Remove all single characters
 
- Remove single characters from the start

- Substitute multiple spaces with single space

- Remove prefixed 'b'

- Convert to Lowercase

In [ ]:
# Your code
import re 

data["text"] = data["text"].apply(lambda x: re.sub(r"[^a-zA-Z0-9\s]", " ", str(x)))

In [ ]:
data["text"] = data["text"].apply(lambda x: re.sub(r"\d+", " ", x))
data

,label,text
0,1,DEAR SIR STRICTLY A PRIVATE BUSINESS PROPOSAL...
1,0,Will do
2,0,Nora Cheryl has emailed dozens of memos about...
3,1,Dear Sir FMadam C I know that this proposal ...
4,0,fyi
...,...,...
995,0,So what s the latest It sounds contradictory ...
996,1,TRANSFER OF MILLION POUNDS TO YOURACCO...
997,0,Barb I will call to explain Are you back in t...
998,0,Yang on travelNot free tonite May work tomorrow


In [122]:
data["text"] = data["text"].apply(lambda x: re.sub(r"\b[a-zA-Z]\b", " ", x))

In [123]:
data["text"] = data["text"].apply(lambda x: re.sub(r"^[a-zA-Z]\s+", "", x))

In [124]:
data["text"] = data["text"].apply(lambda x: re.sub(r"\s+", " ", x).strip())

In [125]:
data["text"] = data["text"].apply(lambda x: re.sub(r"\bb\w*", "", x))


In [126]:
data["text"] = data["text"].apply(lambda x: x.lower())
data


,label,text
0,1,dear sir strictly private business proposal am...
1,0,will do
2,0,nora cheryl has emailed dozens of memos about ...
3,1,dear sir fmadam know that this proposal might ...
4,0,fyi
...,...,...
995,0,so what the latest it sounds contradictory and...
996,1,transfer of million pounds to youraccount my n...
997,0,barb will call to explain are you in the country
998,0,yang on travelnot free tonite may work tomorrow


## Now let's work on removing stopwords
Remove the stopwords.

In [127]:
nltk.download("punkt_tab")

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\JITHIN\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [128]:
print(stopwords.words("english"))

['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't", 'having', 'he', "he'd", "he'll", 'her', 'here', 'hers', 'herself', "he's", 'him', 'himself', 'his', 'how', 'i', "i'd", 'if', "i'll", "i'm", 'in', 'into', 'is', 'isn', "isn't", 'it', "it'd", "it'll", "it's", 'its', 'itself', "i've", 'just', 'll', 'm', 'ma', 'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'shan', "shan't", 'she

In [129]:
# Your code
stop_words = set(stopwords.words("english"))

def remove_stopwords(text):
    tokens = nltk.word_tokenize(str(text))  # tokenize
    filtered = [word for word in tokens if word.lower() not in stop_words]
    return " ".join(filtered)  # join back into string

# Apply to DataFrame
data["text"] = data["text"].apply(remove_stopwords)

# Preview
# print(data[["text", "no_stopwords"]].head())
data

,label,text
0,1,dear sir strictly private business proposal mi...
1,0,
2,0,nora cheryl emailed dozens memos haiti weekend...
3,1,dear sir fmadam know proposal might surprise e...
4,0,fyi
...,...,...
995,0,latest sounds contradictory af decide shall ta...
996,1,transfer million pounds youraccount name mr ej...
997,0,barb call explain country
998,0,yang travelnot free tonite may work tomorrow


## Tame Your Text with Lemmatization
Break sentences into words, then use lemmatization to reduce them to their base form (e.g., "running" becomes "run"). See how this creates cleaner data for analysis!

In [131]:
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize

# Make sure resources are available
nltk.download("punkt")
nltk.download("punkt_tab")   # for tokenization (if missing)
nltk.download("wordnet")     # for lemmatizer
nltk.download("omw-1.4")     # WordNet data


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\JITHIN\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\JITHIN\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\JITHIN\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\JITHIN\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [132]:
nltk.download('averaged_perceptron_tagger')


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\JITHIN\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [133]:
nltk.download('averaged_perceptron_tagger_eng')

# Also make sure you have WordNet for lemmatization
nltk.download('wordnet')
nltk.download('omw-1.4')


[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     C:\Users\JITHIN\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\JITHIN\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\JITHIN\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [134]:
# data["text"] = data["text"].apply(get_wordnet_pos)

In [135]:
lemmatizer = WordNetLemmatizer()

def lemmatize_text(text):
    tokens = word_tokenize(str(text))
    lemmatized = [lemmatizer.lemmatize(word, get_wordnet_pos(word)) for word in tokens]
    return " ".join(lemmatized)

# Apply to your DataFrame
data["text"] = data["text"].apply(lemmatize_text)


In [136]:
data

,label,text
0,1,dear sir strictly private business proposal mi...
1,0,
2,0,nora cheryl email dozen memo haiti weekend ple...
3,1,dear sir fmadam know proposal might surprise e...
4,0,fyi
...,...,...
995,0,late sound contradictory af decide shall take ...
996,1,transfer million pound youraccount name mr eji...
997,0,barb call explain country
998,0,yang travelnot free tonite may work tomorrow


## Bag Of Words
Let's get the 10 top words in ham and spam messages (**EXPLORATORY DATA ANALYSIS**)

In [139]:
# Your code
from collections import Counter

ham_messages = data[data['label'] == 0]['text']
spam_messages = data[data['label'] == 1]['text']

ham_words = [word for message in ham_messages for word in message.split()]
spam_words = [word for message in spam_messages for word in message.split()]

ham_top = Counter(ham_words).most_common(10)
spam_top = Counter(spam_words).most_common(10)

print("Top 10 words in HAM messages:")
print(ham_top)

print("\nTop 10 words in SPAM messages:")
print(spam_top)


Top 10 words in HAM messages:
[('state', 136), ('pm', 127), ('call', 115), ('say', 111), ('work', 107), ('would', 107), ('president', 99), ('time', 95), ('mr', 91), ('obama', 84)]

Top 10 words in SPAM messages:
[('money', 987), ('account', 899), ('fund', 786), ('u', 737), ('transfer', 579), ('transaction', 555), ('country', 513), ('mr', 490), ('million', 463), ('contact', 457)]


## Extra features

In [141]:
# We add to the original dataframe two additional indicators (money symbols and suspicious words).
money_simbol_list = "|".join(["euro","dollar","pound","€",r"\$"])
suspicious_words = "|".join(["free","cheap","sex","money","account","bank","fund","transfer","transaction","win","deposit","password"])

data['money_mark'] = data['text'].str.contains(money_simbol_list)*1
data['suspicious_words'] = data['text'].str.contains(suspicious_words)*1
data['text_len'] = data['text'].apply(lambda x: len(x)) 


data.head()

,label,text,money_mark,suspicious_words,text_len
0,1,dear sir strictly private business proposal mi...,1,1,1452
1,0,,0,0,0
2,0,nora cheryl email dozen memo haiti weekend ple...,0,0,109
3,1,dear sir fmadam know proposal might surprise e...,1,1,1276
4,0,fyi,0,0,3


## How would work the Bag of Words with Count Vectorizer concept?

In [142]:
# Your code
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()


X = vectorizer.fit_transform(data["text"])

bow_df = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names_out())

bow_df['label'] = data['label'].values

ham_words = bow_df[bow_df['label'] == 0].drop('label', axis=1).sum().sort_values(ascending=False).head(10)

spam_words = bow_df[bow_df['label'] == 1].drop('label', axis=1).sum().sort_values(ascending=False).head(10)

print("Top 10 words in HAM messages:")
print(ham_words)

print("\nTop 10 words in SPAM messages:")
print(spam_words)


Top 10 words in HAM messages:
state        136
pm           127
call         115
say          111
work         107
would        107
president     99
time          95
mr            91
obama         84
dtype: int64

Top 10 words in SPAM messages:
money          987
account        899
fund           786
transfer       579
transaction    555
country        513
mr             490
million        463
contact        457
company        420
dtype: int64


## TF-IDF

- Load the vectorizer

- Vectorize all dataset

- print the shape of the vetorized dataset

In [146]:
# Your code
from sklearn.feature_extraction.text import TfidfVectorizer

# Initialize the TF-IDF vectorizer
tfidf_vectorizer = TfidfVectorizer()

# Fit and transform the 'text' column
X_tfidf = tfidf_vectorizer.fit_transform(data['text'])

# Print the shape of the vectorized dataset
print("Shape of TF-IDF matrix:", X_tfidf.shape)


Shape of TF-IDF matrix: (1000, 28160)


## And the Train a Classifier?

In [147]:
# Your code
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

X_train, X_test, y_train, y_test = train_test_split(X_tfidf,data['label'], test_size=0.2, random_state=42)

# Step 2: Initialize the classifier
clf = MultinomialNB()

# Step 3: Train the classifier
clf.fit(X_train, y_train)

# Step 4: Predict on the test set
y_pred = clf.predict(X_test)

# Step 5: Evaluate the model
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))


Accuracy: 0.92

Classification Report:
               precision    recall  f1-score   support

           0       1.00      0.87      0.93       125
           1       0.82      1.00      0.90        75

    accuracy                           0.92       200
   macro avg       0.91      0.94      0.92       200
weighted avg       0.93      0.92      0.92       200


Confusion Matrix:
 [[109  16]
 [  0  75]]


In [148]:
X_train_bag, X_test_bag, y_train_bag, y_test_bag = train_test_split(bow_df.drop(columns="label"),data['label'], test_size=0.2, random_state=42)

# Step 2: Initialize the classifier
clf = MultinomialNB()

# Step 3: Train the classifier
clf.fit(X_train_bag, y_train_bag)

# Step 4: Predict on the test set
y_pred_bag = clf.predict(X_test_bag)

# Step 5: Evaluate the model
print("Accuracy:", accuracy_score(y_test_bag, y_pred_bag))
print("\nClassification Report:\n", classification_report(y_test_bag, y_pred_bag))
print("\nConfusion Matrix:\n", confusion_matrix(y_test_bag, y_pred_bag))

Accuracy: 0.95

Classification Report:
               precision    recall  f1-score   support

           0       1.00      0.92      0.96       125
           1       0.88      1.00      0.94        75

    accuracy                           0.95       200
   macro avg       0.94      0.96      0.95       200
weighted avg       0.96      0.95      0.95       200


Confusion Matrix:
 [[115  10]
 [  0  75]]


### Extra Task - Implement a SPAM/HAM classifier

https://www.kaggle.com/t/b384e34013d54d238490103bc3c360ce

The classifier can not be changed!!! It must be the MultinimialNB with default parameters!

Your task is to **find the most relevant features**.

For example, you can test the following options and check which of them performs better:
- Using "Bag of Words" only
- Using "TF-IDF" only
- Bag of Words + extra flags (money_mark, suspicious_words, text_len)
- TF-IDF + extra flags


You can work with teams of two persons (recommended).

In [ ]:
# Your code